<a href="https://colab.research.google.com/github/Yewon9/Emotion_Recognition_STT/blob/main/STT_Conversion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# IMPORT

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [46]:
!pip install git+https://github.com/openai/whisper.git
!pip install ffmpeg-python

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-otjvjddo
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-otjvjddo
  Resolved https://github.com/openai/whisper.git to commit 25639fc17ddc013d56c594bfbf7644f2185fad84
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 209.5/209.5 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.6 MB/s eta 0:00:00
  Created wheel for openai-whisper: filename=openai_whisper-20240930-py3-none-any.whl size=803321 sha256=a704a1c67fceca47e8d23f87b2cbb8fedd6deb23a85e8dfe06a967ec3005e410
  Stored in directory: /tmp/pip-ephem-wheel-cache-7geeg6ob/wheels/8b/6c/d0/622666868c179f156cf595c8b6f06f88bc5d80c4b31dccaa03
Successfully built openai-whisper


In [47]:
import whisper
import pandas as pd
from tqdm import tqdm

# DATA

In [48]:
data_emotion = pd.read_csv('data_emotion.csv')
data_main = pd.read_csv('data_main.csv')

In [49]:
data_emotion

,Emotion,Path
0,sad,/content/drive/MyDrive/project/data_aihub/5/5e...
1,sad,/content/drive/MyDrive/project/data_aihub/4/5e...
2,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
3,sad,/content/drive/MyDrive/project/data_aihub/5/5f...
4,sad,/content/drive/MyDrive/project/data_aihub/5_2/...
...,...,...
12674,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12675,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12676,happy,/content/drive/MyDrive/project/data_aihub/5_2/...
12677,happy,/content/drive/MyDrive/project/data_aihub/5/5e...


In [50]:
data_main

,Path
0,/content/drive/MyDrive/project/aihub_data_main...
1,/content/drive/MyDrive/project/aihub_data_main...
2,/content/drive/MyDrive/project/aihub_data_main...
3,/content/drive/MyDrive/project/aihub_data_main...
4,/content/drive/MyDrive/project/aihub_data_main...
...,...
409,/content/drive/MyDrive/project/aihub_data_main...
410,/content/drive/MyDrive/project/aihub_data_main...
411,/content/drive/MyDrive/project/aihub_data_main...
412,/content/drive/MyDrive/project/aihub_data_main...


# STT

## emotion

In [51]:
model = whisper.load_model("base")

100%|████████████████████████████████████████| 139M/139M [00:01<00:00, 136MiB/s]
/usr/local/lib/python3.10/dist-packages/whisper/__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this exper

In [53]:
def whisper_transcribe(file_path):
    result = model.transcribe(file_path, language = 'ko')
    return result['text']

In [54]:
def process_multiple_files(file_paths):
    transcriptions = []

    for file_path in tqdm(file_paths, desc = "Processing files"):
        text = whisper_transcribe(file_path)
        transcriptions.append({'Path': file_path, 'STT 결과': text})

    return pd.DataFrame(transcriptions)

In [55]:
file_paths = data_emotion['Path']

In [56]:
emotion_text = process_multiple_files(file_paths)

Processing files:   0%|          | 0/12679 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing files:   0%|          | 1/12679 [00:33<119:22:52, 33.90s/it]/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing files:   0%|          | 2/12679 [00:44<72:00:11, 20.45s/it] /usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")
Processing files:   0%|          | 3/12679 [01:07<74:46:36, 21.24s/it]/usr/local/lib/python3.10/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warning

KeyboardInterrupt: 

## main

### 공백

In [22]:
def get_silence_durations(file_path):
    sound = AudioSegment.from_wav(file_path)

    silence_ranges = detect_silence(sound, min_silence_len = 500, silence_thresh = -40)

    silence_durations = [(end - start) for start, end in silence_ranges]

    return silence_durations

In [26]:
def calculate_silence_stats_with_tqdm(data_paths):
    all_silences = []

    for file_path in tqdm(data_paths, desc="Processing files"):
        silences = get_silence_durations(file_path)
        all_silences.extend(silences)

    if all_silences:
        avg_silence = sum(all_silences) / len(all_silences)
        min_silence = min(all_silences)
        return avg_silence / 1000, min_silence / 1000
    else:
        return 0, 0

In [30]:
average_silence, min_silence = calculate_silence_stats_with_tqdm(data_main['Path'])

Processing files:   2%|▏         | 7/414 [07:23<7:09:49, 63.36s/it]


KeyboardInterrupt: 

In [31]:
plt.figure(figsize=(10, 6))
plt.hist(all_silences, bins=20, color='blue', alpha=0.7, edgecolor='black')
plt.axvline(average_silence, color='red', linestyle='dashed', linewidth=1, label=f'Average Silence: {average_silence:.2f}s')
plt.axvline(min_silence, color='green', linestyle='dashed', linewidth=1, label=f'Min Silence: {min_silence:.2f}s')
plt.title('Distribution of Silence Durations')
plt.xlabel('Silence Duration (Seconds)')
plt.ylabel('Frequency')
plt.legend()
plt.show()

average_silence, min_silence

NameError: name 'all_silences' is not defined

<Figure size 1000x600 with 0 Axes>

### STT - 50개씩 나눠서

In [39]:
recognizer = sr.Recognizer()

In [40]:
def split_audio_on_silence(file_path):
  sound = AudioSegment.from_wav(file_path)

  chunks = split_on_silence(sound, min_silence_len = 1000, silence_thresh = -40)
  return chunks

In [41]:
def speech_to_text(audio_chunk):
    try:
        audio_data = audio_chunk.export(format = "wav")
        with sr.AudioFile(audio_data) as source:
            audio = recognizer.record(source)
            text = recognizer.recognize_google(audio, language = "ko-KR")
            return text
    except sr.UnknownValueError:
        return "음성을 인식할 수 없음"
    except sr.RequestError as e:
        return f"STT API 오류: {e}"

In [42]:
def process_audio(file_path):
    chunks = split_audio_on_silence(file_path)
    text_results = []

    for i, chunk in enumerate(chunks):
        text = speech_to_text(chunk)
        text_results.append(f"Chunk {i+1}: {text}")

    return text_results

In [43]:
data_list = []
for file_path in tqdm(data_main['Path'][:3], desc = "Processing 10 files"):
    stt_results = process_audio(file_path)
    for result in stt_results:
        data_list.append({'Path': file_path, 'STT 결과': result})

Processing 10 files: 100%|██████████| 10/10 [21:49<00:00, 130.99s/it]


In [44]:
data_main = pd.DataFrame(data_list)
data_main

,Path,STT 결과
0,/content/drive/MyDrive/project/aihub_data_main...,Chunk 1: 미영
1,/content/drive/MyDrive/project/aihub_data_main...,Chunk 2: 아이
2,/content/drive/MyDrive/project/aihub_data_main...,Chunk 3: 단짝 미애와 아이는 노래하며 웁니다
3,/content/drive/MyDrive/project/aihub_data_main...,Chunk 4: 나를 위해 나란히 누워
4,/content/drive/MyDrive/project/aihub_data_main...,Chunk 5: 랑랑이 노래합니다
...,...,...
985,/content/drive/MyDrive/project/aihub_data_main...,Chunk 95: 나는 지난주 목요일에 인터넷으로 원피스를 주문했다 내일 친구 생일...
986,/content/drive/MyDrive/project/aihub_data_main...,Chunk 96: 음성을 인식할 수 없음
987,/content/drive/MyDrive/project/aihub_data_main...,Chunk 97: 음성을 인식할 수 없음
988,/content/drive/MyDrive/project/aihub_data_main...,Chunk 98: 음성을 인식할 수 없음


In [45]:
data_main.to_csv('main_text.csv', index = False, encoding = 'utf-8-sig')